In [2]:
import sys
from pprint import pprint
import os
from bs4 import BeautifulSoup
import re
import traceback
from unidecode import unidecode
import mpmath

# For python code form edX
import random

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
hw0_vertical_names = [
    '60fb25d0e9f0487dad5d91a1ef501fd9',
    '908da8a3abda4a019e8e42416ae61555',
    '614c12e8de7746399f84b1967b0b807a',
    '90f45f0a8b994990b638f2499f0c8559',
    '481bd431910848d1bfd717b775f3ab30',
    '7b03a0b7e96244db86c6ea2b93387fea',
    'fbd9801aaf464cfd92ee5bda74c00a69',
]

hw3_vertical_names = [
    '466c91e2a2cb432782905665ef2ffc7b',
    'fd16b392949f415ebffea55f27d80d44',
    'a309502f50d74b9c894dcefa3c473d17',
    'f9396e26808a4110b31c5b8b3c8079a3',
    '3712f650c3b74c1398eca1c5bca61cdc',
    'de5922b4818f41a09e815342a0880097',
    '301d9eee9a374f8a8766f526be761aac',
    '85e9ac7604d542329b2f00030769fbdb',
    'e04115d5fcce47158de7e19e64124c77',
    '5c7aa00d3dad42c1bbc759621ed0f5e9',
    'e0fd8518bede418daf4867418f59b642',
    'aa501fff0f54496283836560ab1ef8ab',
]

hw4_vertical_names = [
    'af09c0b67c1c4c3c9a4bfb411752e378',
    'c42308f0d45243a991f3a0092463117e',
    '13c0d2e57ad14de189eaf679185d919e',
    '80b7d3e5a5964671a2d01f0c5d3fcfbe',
    '38cb5f04a32b4b0c9efa88204fc4ac93',
    'ed2acf12087e41088ff14d8538f78dee',
    '7d455ad5de1644e1af154dc040e9ea86',
    'bee7b6f3c85d4058921a1dc89c628ba9',
    '66779dca33cb42dab94191fe3977e68b',
    'bff51633d2cb4db1acd7a0ad3725edb1',
    'de38506701cc495f9d2d3c956250f93d',
]

hw5_vertical_names = [
    '12737afb6a294563a20c44abaed2bdcb',
    'a6d782741a20484ca2935f3035d31f75',
    '847a056cef4b4b01bf603be6ad863c2a',
    '5c313864870f446d9bf73b2fc76cdb50',
    'd1d68fb420e7430f87e38458adbdc628',
    '1f2df8c51a634e20a19bf4d2bb28451e',
    'b20ce240ad534a4192c993275991a7dc',
    '2bdfcc1f50a546378fc451857a257f2a',
    '18877010412d4c2ca59833c5842c3308',
]

hw6_vertical_names = [
    '4b3a4606776e4445a8a6a44a45592b7a',
    'dbe4e756753841f09f08506fecac2455',
    'd5b901667f274a7c8caf0d2385cea312',
    'd8f9ba3156b14b7fab8925ecd7f27126',
    'a9bbca9cf0df4d409b6a12de9a2bb5bf',
    'efa79f27347d42d4962ef32405b9e5f2',
]

hw7_vertical_names = [
    'b28bfbe219ff4c00848bdcd093a94317',
    '8cb39eb21b6b42b7b1e94c65f49fe07f',
    '406a5999cde84e029ac335ece5971e22',
    '63313c514b4242da8d5b7eebc14000ee',
    '041a6bfbf8024cf0b8dd1c9c1e9dd314',
    '07d1378020734acfa56a8fc4c5014c2b',
    'c8600e8c0cb448a9b1f7d7fe620085fd',
    '6eb8b298e8df43ba8a31a92a96718b46',
    '5bfd16d9d2884ee68a047bebced40464',
    '8838893f726647f394322340a4dd16ab',
    'eef82b7ccbee4ca096916b4ae389c7d8',
    '8f7c9a96107342218b6eaec2309078f1',
    '97c682bb5e9b42568f264599df8b0400',
]

hw8_vertical_names = [
    '309f32b88a0a4ba68668d834ec7f7c51',
    'dd5b2de9d8ed4cf995a9af22cfdaaf4a',
    'd193b62a8ae5480583c416bcc7f7125e',
    '445ad9c4dfb646c2a0013c3ec1178d69',
    '72518c2c80c140c982da3bd92037d5a7',
    'aad34254de554e46a0e9c745e1955738',
]

hw9_vertical_names = [
    'e56708a2334e402495d61a0d0f68d64c',
    '76fe6de5c3714a1f996b8c711b65fd10',
    'f62990ed899f441dbdadb1b4ecefb687',
    'e6af13845b7849d585de93d593210f0f',
    'ebf7ba851bdb4dcaa0c8debc5f028d40',
    '9d0cebe29f12468fb7176f187789a959',
    'c2b39f5075e4445790d55cd5d12d617a',
]

hw10_vertical_names = [
    '6dce0fa2203f4fe9adf415907c9d49fb',
    '359be81466854ef9ac4993b619c03ad2',
]

In [4]:
class TableIndexCounter:
    def __init__(self):
        self.table_index = 0

class TableEntryProcessor:
    def __init__(self, tic):
        self.answers = {}
        self.nested_tables = {}
        self.tic = tic
    
    def process(self, tag):
        table = tag.find('table')
        textline = tag.find('textline')
        if table:
            text = self.process_nested_table(table)
            return text
        elif textline:
            if not textline.has_attr('correct_answer'):
                return ''
            else:
                correct_answer_text, correct_answer_value, points = convert_textline_format(textline)
                self.answers[correct_answer_text] = {
                    'value': correct_answer_value,
                    'points': points,
                }
                return '**' + correct_answer_text + '**'
        else:
            text = re.sub('\$\w+', self.process_match, tag.text)
            text = convert_latex_format(text.strip())
            return text
        
    def process_nested_table(self, table):
        self.nested_tables[self.tic.table_index] = convert_table_format(table, self.tic)
        text = 'Table' + str(self.tic.table_index)
        self.tic.table_index += 1
        return text
        
    def process_match(self, match):
        text = '{' + match.group(0)[1:] + '}'
        text = text.format(**globals())
        try:
            # This still doesn't work
            text = str(float(str(text)))
        except:
            pass
        if '</table>' in text:
            table = BeautifulSoup(text, 'html.parser').find('table')
            text = self.process_nested_table(table)
        return text


In [5]:
def evaluate_variable(match):
    matched_text = match.group(0)[1:]
    text = '{' + matched_text + '}'
    try:
        text = text.format(**globals())
    except KeyError:
        pass
    return text


def evaluate_variables(text):
    return re.sub('\$\w+', evaluate_variable, text)


def convert_latex_format(text):
    return text.replace('\(', '$$').replace('\)', '$$').replace('\[', '$$').replace('\]', '$$').replace('|', ' \mid ')


def convert_img_format(img_src):
    return '![](https://rawgit.com/AdityaB97/cs188/master{})'.format(evaluate_variables(img_src))


def convert_numericalresponse_format(answer):
    return '[____]({})'.format(evaluate_variables(answer))


def convert_choiceresponse_format(tag):
    result = ''
    multichoice = sum([choice['correct'] == 'true' for choice in tag.find_all('choice')]) > 1
    for choice in tag.find_all('choice'):
        if multichoice:
            prefix = '[X] ' if choice['correct'] == 'true' else '[ ] '
        else:
            prefix = '(X) ' if choice['correct'] == 'true' else '( ) '
        result += prefix
        if choice.find('text') is not None:
            result += choice.find('text').text.strip() + '\n'
        else:
            # List comprehension below is a quick fix for HW10 Q6 (actually second question)
            result += ' '.join([str(thing) for thing in choice.contents]) + '\n'
    return result


def convert_textline_format(textline):
    assert textline.has_attr('correct_answer')
    correct_answer_text = re.search('\$\w+', textline['correct_answer'])
    # If the correct answer is a variable, evaluate it. Else, use the raw value
    if correct_answer_text:
        correct_answer_text = correct_answer_text.group(0)[1:]
    else:
        correct_answer_text = textline['correct_answer']
    try:
        correct_answer_value = float(str(eval(correct_answer_text)))
    except Exception as e:
        correct_answer_value = str(eval(correct_answer_text))
    # points = textline['points']
    # Uncomment if points is needed
    points = None
    # Capitalize Answer to make it look better
    correct_answer_text = correct_answer_text.capitalize()
    
    return correct_answer_text, correct_answer_value, points
    

def convert_table_format(table, tic):
    tep = TableEntryProcessor(tic)
    output_string = ''
    
    if table.find('tbody'):
            table = table.find('tbody')
            
    for tag in table.contents:
        if tag.name == 'tr':
            # Hacky solution for tables within tables
            # Removed because this skips tables in variables, like in HW6 Q1
            # if tag.find('table'):
            #     continue
            
            th_tags = tag.find_all('th', recursive=False)
            td_tags = tag.find_all('td', recursive=False)

            # Removed temporarily because this fails when there are nested tables
            # But then resolved the issue by making recursive=False above 
            assert (len(th_tags) > 0) ^ (len(td_tags) > 0), (th_tags, td_tags)

            if len(th_tags) > 0:
                output_string += '| ' + ' '.join([tep.process(th_tag) + ' |' for th_tag in th_tags]) + '\n'
                output_string += '| ' + ' --- |' * len(th_tags) + '\n'
            elif len(td_tags) > 0:
                output_string += '| ' + ' '.join([tep.process(td_tag) + ' |' for td_tag in td_tags]) + '\n'
                
    output_string += '\n$$\\phantom{\\rule{0em}{1em}}$$'
    output_string += '\n\n'
    if len(tep.answers) > 0:
        output_string += '### Input Answers Here \n'
    for answer_name, answer_dict in sorted(tep.answers.items(), key=lambda x : x[0]):
        value, points = answer_dict['value'], answer_dict['points']
        output_string += '**' + answer_name + ':**' + '\n'
        output_string += '[____]({})'.format(value) + '\n\n'
        
    for index, table in sorted(tep.nested_tables.items(), key=lambda x : x[0]):
        output_string += '**Table' + str(index) + ':**' + '\n\n'
        output_string += table + '\n\n'
        
    return output_string


def convert_tag(tag, tic):
    if tag.name == 'table':
        text = convert_table_format(tag, tic)
        text += '\n\n'
        return text
    elif tag.name is None:
        tag_text = str(unidecode(tag)).strip()
        tag_text_evaluated = evaluate_variables(tag_text)
        if tag_text != tag_text_evaluated:
            return convert_tag(BeautifulSoup(tag_text_evaluated, 'html.parser'), tic) + '\n\n'
        else:
            return convert_latex_format(tag_text) + '\n\n'
    elif tag.name == 'problem':
        if tag.has_attr('weight'):
            weight = tag['weight']
        else:
            weight = 1
        text = tag['display_name'] + ' ' + str(weight)
        text += '\n\n'
    elif tag.name == 'img':
        text = convert_img_format(tag['src'])
        text += '\n\n'
    elif tag.name == 'numericalresponse' or tag.name == 'stringresponse':
        text = convert_numericalresponse_format(tag['answer'])
        text += '\n\n'
    elif tag.name == 'choiceresponse' or tag.name == 'multiplechoiceresponse':
        text = convert_latex_format(convert_choiceresponse_format(tag))
        text += '\n\n'
        return text
    elif tag.name == 'textline':
        if not tag.has_attr('correct_answer'):
            text = ''
        else:
            correct_answer_text, correct_answer_value, points = convert_textline_format(tag)
            text = '**' + correct_answer_text + ':**' + '\n'
            text += '[____]({})'.format(correct_answer_value)
            text += '\n\n'
    elif (tag.has_attr('type') and tag['type'] == 'loncapa/python') or tag.name in ['style']:
        return ''
    else:
        text = ''
        
    return text + ''.join([convert_tag(child, tic) for child in tag.children if child != '\n'])

In [6]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        
def format_python_code(text):
    return text.replace('&gt;', '>').replace('&lt;', '<').replace('&amp;', '&')

In [8]:
vertical_names = hw6_vertical_names

for vertical_name in vertical_names:
    path_to_vertical = find('{}.xml'.format(vertical_name), './course_sp18/')
    with open(path_to_vertical) as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    vertical_display_name = soup.find_all('vertical')[0]['display_name']
    problem_url_names_and_types = [(tag['url_name'], tag.name) for tag in soup.find_all(['problem', 'html'])]
    for part_index, problem_url_name_and_type in enumerate(problem_url_names_and_types):
        problem_url_name, problem_url_type = problem_url_name_and_type
        if problem_url_type == 'problem':
            path_to_problem = find('{}.xml'.format(problem_url_name), './course_sp18/')
        elif problem_url_type == 'html':
            path_to_problem = find('{}.html'.format(problem_url_name), './course_sp18/')
        else:
            raise Exception('fuck up')
        with open(path_to_problem) as f:
            problem_soup = BeautifulSoup(f, 'html.parser')

        for tag in problem_soup.find_all('solution'):
            tag.clear()

        script = problem_soup.find(['script'])
        if script:
            try:
                exec(format_python_code(script.text), globals())
            except Exception as e:
                print(e)
                print(traceback.format_exc())
                print('ERROR WITH EXECUTING CODE: ', problem_url_name)
        
        tic = TableIndexCounter()
        try:
            problem_text = convert_tag(problem_soup, tic)
        except Exception as e:
            print(e)
            print(traceback.format_exc())
            print('ERROR WITH TRANSLATING: ', problem_url_name)
        
        if part_index != 0:
            print '---------------------\n\n'
            print 'Part {}'.format(part_index + 1)
        print problem_text
    
    print('\n$$\\phantom{\\rule{0em}{10em}}$$')
    print('___')
    print '\n\n\n--------------------------------------------------------------------\n\n\n'
    


Question 1: Probability, Part I 16.0

Below is a table listing the probabilities of three binary random variables.
In the empty table cells, fill in the correct values for each marginal or conditional probability.

Your answers will be evaluated to 4 decimal places, so you may find it easier to enter answers as expressions such as $$0.14*0.08/2$$.
The grader will evaluate these expressions maintaining full floating point precision.

| Table0 | Table1 |

$$\phantom{\rule{0em}{1em}}$$

**Table0:**

| $$X_0$$ | $$X_1$$ | $$X_2$$ | $$P(X_0,X_1,X_2)$$ |
|  --- | --- | --- | --- |
| 0 | 0 | 0 | 0.220 |
| 1 | 0 | 0 | 0.100 |
| 0 | 1 | 0 | 0.060 |
| 1 | 1 | 0 | 0.080 |
| 0 | 0 | 1 | 0.100 |
| 1 | 0 | 1 | 0.140 |
| 0 | 1 | 1 | 0.140 |
| 1 | 1 | 1 | 0.160 |

$$\phantom{\rule{0em}{1em}}$$



**Table1:**

| Expression | Value |
|  --- | --- |
| $$P(X_0=1,X_1=0,X_2=1)$$ | **Answer0** |
| $$P(X_0=0,X_1=1)$$ | **Answer1** |
| $$P(X_2=0)$$ | **Answer2** |
| $$P(X_1=0  \mid  X_0=1)$$ | **Answer3** |
| 

Question 6: Bayes' Nets Independence 18.0

Let $$H_x$$ be a random variable denoting the handedness of an individual $$x$$, with possible values $$l$$ or $$r$$. A common hypothesis is that left- or right-handedness is inherited by a simple mechanism; that is, perhaps there is a gene $$G_x$$, also with values $$l$$ or $$r$$, and perhaps actual handedness turns out mostly the same (with some probability $$s$$) as the gene an individual possesses. Furthermore, perhaps the gene itself is equally likely to be inherited from either of an individual's parents, with a small nonzero probability $$m$$ of a random mutation flipping the handedness.

![](https://rawgit.com/AdityaB97/cs188/masterhttps://edge.edx.org/c4x/BerkeleyX/CS188-FA14/asset/14-20a.png)

![](https://rawgit.com/AdityaB97/cs188/masterhttps://edge.edx.org/c4x/BerkeleyX/CS188-FA14/asset/14-20b.png)

![](https://rawgit.com/AdityaB97/cs188/masterhttps://edge.edx.org/c4x/BerkeleyX/CS188-FA14/asset/14-20c.png)

Which of the three netwo

# Rough work

In [79]:
problem_url_name = '5e64c08964794920b2c07690d70908f8'
part_index = 0

path_to_problem = find('{}.xml'.format(problem_url_name), './course_sp18/')
with open(path_to_problem) as f:
    problem_soup = BeautifulSoup(f, 'html.parser')

for tag in problem_soup.find_all('solution'):
    tag.clear()

script = problem_soup.find(['script'])
if script:
    try:
        exec(format_python_code(script.text), globals())
    except Exception as e:
        print(e)
        print(traceback.format_exc())
        print('ERROR WITH EXECUTING CODE: ', problem_url_name)

try:
    list_of_contents = [convert_format(tag) for tag in problem_soup.find_all(['problem', 'p', 'img', 'numericalresponse', 'choiceresponse', 'table', 'text', 'multiplechoiceresponse', 'stringresponse'])]
except Exception as e:
    print(e)
    print(traceback.format_exc())
    print('ERROR WITH TRANSLATING: ', problem_url_name)
problem_text = '\n\n'.join(list_of_contents)

if part_index != 0:
    print '---------------------\n\n'
    print 'Part {}'.format(i + 1)
print problem_text    


Question 1: Probability, Part I 16.0

Below is a table listing the probabilities of three binary random variables.
In the empty table cells, fill in the correct values for each marginal or conditional probability.

Your answers will be evaluated to 4 decimal places, so you may find it easier to enter answers as expressions such as $$0.14*0.08/2$$.
The grader will evaluate these expressions maintaining full floating point precision.

| 
    table0
   | answer0 |


answer0:
[____](0.14)

table0:
| $$X_0$$ | $$X_1$$ | $$X_2$$ | $$P(X_0,X_1,X_2)$$ |
|  --- | --- | --- | --- |
|  0 |  0 |  0 | 0.100 |
|  1 |  0 |  0 | 0.180 |
|  0 |  1 |  0 | 0.080 |
|  1 |  1 |  0 | 0.120 |
|  0 |  0 |  1 | 0.080 |
|  1 |  0 |  1 | 0.140 |
|  0 |  1 |  1 | 0.080 |
|  1 |  1 |  1 | 0.220 |






|  Expression  | Value |
|  --- | --- |
|  $$P(X_0=1,X_1=0,X_2=1)$$  | answer0 |
|  $$P(X_0=0,X_1=1)$$  | answer1 |
|  $$P(X_2=0)$$  | answer2 |
|  $$P(X_1=0  \mid  X_0=1)$$  | answer3 |
|  $$P(X_0=1,X_1=0  \mid  X_

In [77]:
convert_table_format(problem_soup.find('table'))

u'| \n    table0\n   | answer0 |\n\n\nanswer0:\n[____](0.12)\n\ntable0:\n| $$X_0$$ | $$X_1$$ | $$X_2$$ | $$P(X_0,X_1,X_2)$$ |\n|  --- | --- | --- | --- |\n|  0 |  0 |  0 | 0.100 |\n|  1 |  0 |  0 | 0.120 |\n|  0 |  1 |  0 | 0.080 |\n|  1 |  1 |  0 | 0.240 |\n|  0 |  0 |  1 | 0.080 |\n|  1 |  0 |  1 | 0.120 |\n|  0 |  1 |  1 | 0.180 |\n|  1 |  1 |  1 | 0.080 |\n\n\n\n\n'

In [76]:
problem_soup.find('table')

<table align="center">\n<tr>\n<td class="numbers">\n    $html\n  </td>\n<td>\n<div class="choice_table">\n<table class="dearlord">\n<tr>\n<th> Expression </th>\n<th>Value</th>\n</tr>\n<tr>\n<td class="expr"> $text0 </td>\n<td>\n<textline correct_answer="\\($answer0\\)" points="1" size="9"></textline>\n</td>\n</tr>\n<tr>\n<td class="expr"> $text1 </td>\n<td>\n<textline correct_answer="\\($answer1\\)" points="1" size="9"></textline>\n</td>\n</tr>\n<tr>\n<td class="expr"> $text2 </td>\n<td>\n<textline correct_answer="\\($answer2\\)" points="1" size="9"></textline>\n</td>\n</tr>\n<tr>\n<td class="expr"> $text3 </td>\n<td>\n<textline correct_answer="\\($answer3\\)" points="1" size="9"></textline>\n</td>\n</tr>\n<tr>\n<td class="expr"> $text4 </td>\n<td>\n<textline correct_answer="\\($answer4\\)" points="1" size="9"></textline>\n</td>\n</tr>\n<tr>\n<td class="expr"> $text5 </td>\n<td>\n<textline correct_answer="\\($answer5\\)" points="1" size="9"></textline>\n</td>\n</tr>\n</table>\n</div>\n